In [1]:
import gc
import os
import warnings
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

import tensorflow as tf
import tensorflow_probability as tfp

from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, GlobalAveragePooling2D, BatchNormalization, Input
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import categorical_crossentropy

import cv2
import PIL
from PIL import ImageOps, ImageFilter, ImageDraw

warnings.filterwarnings(action='ignore')

In [2]:
CROP_PATH = './image/'
DATA_PATH = './data/'

# image folder path
TRAIN_IMG_PATH = CROP_PATH +'/train_crop/'
TEST_IMG_PATH = CROP_PATH +'/test_crop/'

# read csv
df_train = pd.read_csv(DATA_PATH + '/train.csv')
df_train = df_train.iloc[:9990]
df_test = pd.read_csv(DATA_PATH + '/test.csv')
df_class = pd.read_csv(DATA_PATH + '/class.csv')

from sklearn.model_selection import train_test_split

df_train["class"] = df_train["class"].astype('str')

df_train = df_train[['img_file', 'class']]
df_train.replace('196', '0', inplace = True)
df_test = df_test[['img_file']]

# import matplotlib.image as mpimg

# graycolors = []

# for i in df_train['img_file']:
#     try:
#         img = mpimg.imread('./image/train_crop/' + i)
#     except:
#         index = df_train.loc[df_train['img_file'] == i].index
#         df_train.drop(index, inplace=True)

x_train = df_train['img_file'].to_numpy()
y_train = df_train['class'].astype('int').to_numpy()

In [3]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def create_base_model(base_model):
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, kernel_initializer='he_normal')(x)
    x = Dropout(0.3)(x)
    x = Activation('relu')(x)
    outputs = Dense(196, activation = 'softmax')(x)

    model = Model(inputs = base_model.input, outputs = outputs)
    
    return model

In [15]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))

def preprocessing(image_path, label):
    img = tf.io.read_file('./image/train_crop/' + image_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [299, 299])
    
    return img, label

BATCH_SIZE = 16

train_ds = train_ds.map(preprocessing)
train_ds = train_ds.shuffle(32).batch(BATCH_SIZE).repeat()
# train_ds = train_ds.repeat().prefetch(tf.data.experimental.AUTOTUNE)

In [5]:
from tensorflow.keras.optimizers import Adam
lr = 1e-4
optimizer = Adam(learning_rate = lr)

from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
loss_fn = CategoricalCrossentropy()

from tensorflow.keras.metrics import SparseCategoricalAccuracy
train_acc_metric = SparseCategoricalAccuracy()

In [12]:
def rand_bbox(size, lam):
    W = size[1]
    H = size[2]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

@tf.function
def train_step(x, target_a, target_b, lam):
    with tf.GradientTape() as tape:
        outputs = model(x, training = True)
        loss_value = loss_fn(target_a, outputs) * lam + loss_fn(target_b, outputs) * (1. - lam)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
#     train_acc_metric.update_state(target_a, outputs)
    return loss_value

# @tf.function
# def train_step(x, y):
#     with tf.GradientTape() as tape:
#         outputs = model(x, training = True)
#         outputs += 1e-10
# #         tf.print(outputs)
#         loss_value = loss_fn(y, outputs)
#     grads = tape.gradient(loss_value, model.trainable_weights)
#     optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
#     return loss_value

In [10]:
base_model = tf.keras.applications.MobileNetV2(weights = 'imagenet', input_shape = (299, 299, 3), include_top = False)
model = create_base_model(base_model)

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        lam = tfp.distributions.Beta(1, 1).sample()
        rand_index = tf.random.shuffle(tf.range(16))
        target_a = labels
        target_a = tf.keras.utils.to_categorical(target_a, num_classes=196)
        target_b = tf.gather(labels, rand_index)
        target_b = tf.keras.utils.to_categorical(target_b, num_classes=196)
        bbx1, bby1, bbx2, bby2 = rand_bbox(images.shape, lam)
        
        images = images.numpy()
        shuffled_images = tf.gather(images, rand_index).numpy()
        images[:, bbx1:bbx2, bby1:bby2, :] = shuffled_images[:, bbx1:bbx2, bby1:bby2, :]
        images = tf.convert_to_tensor(images)
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (images.shape[-2] * images.shape[-3]))

        loss_value = train_step(images, target_a, target_b, lam)
#         labels = tf.keras.utils.to_categorical(labels, num_classes=196)
        
#         loss_value = train_step(images, labels)
        tf.print(loss_value)
    
    train_acc = train_acc_metric.result()
    print(f"Training acc over epoch: {float(train_acc)}")
    train_acc_metric.reset_states()

W0520 01:13:29.474012  4584 def_function.py:586] 11 out of the last 11 calls to <function train_step at 0x0000022338055840> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


5.04703045


W0520 01:13:32.563702  4584 def_function.py:586] 11 out of the last 11 calls to <function train_step at 0x0000022338055840> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


5.07894516


W0520 01:13:35.668191  4584 def_function.py:586] 11 out of the last 11 calls to <function train_step at 0x0000022338055840> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


...

5.36218262


W0520 01:29:46.416315  4584 def_function.py:586] 11 out of the last 11 calls to <function train_step at 0x0000022338055840> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


5.32064


W0520 01:29:51.436039  4584 def_function.py:586] 11 out of the last 11 calls to <function train_step at 0x0000022338055840> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


5.34893179
